# LivenProteins — Bioreactor Optimization (Template Rebuild)

**Purpose:** Predict **OD600** and **Collagen** from process parameters (DO, Airflow sL/h, Agitation rpm, pH, Feed rate/total, residual glycerol/methanol).  



In [62]:
# === Global paths ===
OUTPUT_DIR = Path("optinome_outputs")
OUTPUT_DIR.mkdir(exist_ok=True)
# === Setup ===
from pathlib import Path

# 👉 CHANGE THIS TO YOUR REAL PROJECT FOLDER (this matches your terminal)
PROJECT_DIR = Path("/Users/peyma/Desktop/PROJECT 24-0048 (CII Prototype Strain Bioreactor Validation)")

# If all your EXP decks & summary files are already in this folder (or subfolders):
EXTRACT_DIR = PROJECT_DIR  # we just walk the project tree directly

# Where plots and outputs will be saved
OUTPUT_DIR = PROJECT_DIR / "outputs"
OUTPUT_DIR.mkdir(exist_ok=True)

print("PROJECT_DIR:", PROJECT_DIR)
print("EXTRACT_DIR:", EXTRACT_DIR)
print("OUTPUT_DIR:", OUTPUT_DIR)


PROJECT_DIR: /Users/peyma/Desktop/PROJECT 24-0048 (CII Prototype Strain Bioreactor Validation)
EXTRACT_DIR: /Users/peyma/Desktop/PROJECT 24-0048 (CII Prototype Strain Bioreactor Validation)
OUTPUT_DIR: /Users/peyma/Desktop/PROJECT 24-0048 (CII Prototype Strain Bioreactor Validation)/outputs


In [63]:
# === Helpers ===
def iter_excels(root: Path, priority_names: set, limit: int):
    files = []
    for r, _, fs in os.walk(root):
        for f in fs:
            if f.lower().endswith(('.xls', '.xlsx')):
                files.append(Path(r)/f)
    files = sorted(files, key=lambda p: (0 if p.name in priority_names else 1, p.name))
    return files[:limit]

def get_time_hours(series):
    s = series
    if pd.api.types.is_numeric_dtype(s):
        return pd.to_numeric(s, errors='coerce')
    dt = pd.to_datetime(s, errors='coerce')
    if dt.notna().sum() >= 2:
        return (dt - dt.min()) / np.timedelta64(1,'h')
    num = pd.to_numeric(s, errors='coerce')
    if num.notna().sum() >= 2:
        return num.astype(float)
    return pd.Series(np.arange(len(s)), index=s.index, dtype=float)

re_do_pv = re.compile(r"DO\s*\d+\.PV\s*\[\s*%DO\s*\]", re.IGNORECASE)
re_do_sp = re.compile(r"DO\.SP\[\s*\d+\s*\]\s*\[\s*%DO\s*\]", re.IGNORECASE)
re_fa_pv = re.compile(r"FA\s*\d+\.PV\s*\[\s*mL/h\s*\]", re.IGNORECASE)

def detect_col(cols, tests):
    for t in tests:
        for c in cols:
            if t(c):
                return c
    return None

def parse_growth(xf: ExcelFile):
    sh = next((s for s in xf.sheet_names if s.strip().lower()=='growth'), None)
    if sh is None: return None
    df = xf.parse(sh)
    if df.empty: return None
    df.columns = [str(c).strip() for c in df.columns]
    cols = list(df.columns)
    time_col = detect_col(cols, [lambda c: any(k in c.lower() for k in ['timestamp','time','duration','elapsed','hour','hr','date'])])
    od_col   = detect_col(cols, [lambda c: 'od600' in c.lower() or re.search(r"\bod\s*600\b", c, flags=re.IGNORECASE)])
    do_pv_col = detect_col(cols, [lambda c: re_do_pv.search(c) is not None])
    do_sp_col = detect_col(cols, [lambda c: re_do_sp.search(c) is not None])
    fa_pv_col = detect_col(cols, [lambda c: re_fa_pv.search(c) is not None])
    t = get_time_hours(df[time_col]) if time_col else pd.Series(np.arange(len(df)))
    return pd.DataFrame({'time_hours': t,
                         'od600': pd.to_numeric(df[od_col], errors='coerce') if od_col in df.columns else np.nan,
                         'do_pv': pd.to_numeric(df[do_pv_col], errors='coerce') if do_pv_col in df.columns else np.nan,
                         'do_sp': pd.to_numeric(df[do_sp_col], errors='coerce') if do_sp_col in df.columns else np.nan,
                         'feed_ml_h': pd.to_numeric(df[fa_pv_col], errors='coerce') if fa_pv_col in df.columns else np.nan})

def col_letter_idx(letter: str) -> int:
    total=0
    for ch in letter.upper():
        total=total*26+(ord(ch)-ord('A')+1)
    return total-1

def parse_data(xf: ExcelFile):
    sh = next((s for s in xf.sheet_names if s.strip().lower()=='data'), None)
    if sh is None:
        sh = next((s for s in xf.sheet_names if 'data' in s.strip().lower()), None)
    if sh is None: return None
    df = xf.parse(sh, header=0)
    if df.empty: return None
    df.columns = [str(c).strip() for c in df.columns]
    cols = list(df.columns)
    time_col = detect_col(cols, [lambda c: any(k in c.lower() for k in ['timestamp','time','duration','elapsed','hour','hr','date'])])
    t = get_time_hours(df[time_col]) if time_col in df.columns else get_time_hours(df.iloc[:,0])
    def get_by_idx(i):
        return pd.to_numeric(df.iloc[:, i], errors='coerce') if i<df.shape[1] else pd.Series([np.nan]*len(df), index=df.index)
    return pd.DataFrame({'time_hours': t,
                         'do_data': get_by_idx(col_letter_idx('F')),
                         'air_slph': get_by_idx(col_letter_idx('G')),
                         'agitation_rpm': get_by_idx(col_letter_idx('O')),
                         'ph_data': get_by_idx(col_letter_idx('Q'))})

def parse_hplc(xf: ExcelFile):
    sh = next((s for s in xf.sheet_names if 'hplc' in s.strip().lower()), None)
    if sh is None: return None
    df = xf.parse(sh)
    if df.empty: return None
    df.columns = [str(c).strip() for c in df.columns]
    cols = list(df.columns)
    time_col = detect_col(cols, [lambda c: any(k in c.lower() for k in ['timestamp','time','sample time','collection time','date','elapsed','hour','hr'])])
    t = get_time_hours(df[time_col]) if time_col in df.columns else get_time_hours(df.iloc[:,0])
    gly_col = detect_col(cols, [lambda c: ('glycerol' in c.lower() or 'glyc' in c.lower() or 'gly' in c.lower()) and any(u in c.lower() for u in ['g/l','mg/l','conc','concentration','area','peak'])])
    meoh_col= detect_col(cols, [lambda c: ('methanol' in c.lower() or 'meoh' in c.lower()) and any(u in c.lower() for u in ['g/l','mg/l','conc','concentration','area','peak'])])
    return pd.DataFrame({'time_hours': t,
                         'glycerol': pd.to_numeric(df[gly_col], errors='coerce') if gly_col in df.columns else np.nan,
                         'methanol': pd.to_numeric(df[meoh_col], errors='coerce') if meoh_col in df.columns else np.nan})

def parse_compilation(xf: ExcelFile, name: str):
    if name.lower()!='data_compilation.xlsx':
        return None
    sh = xf.sheet_names[0]
    df = xf.parse(sh)
    if df.empty: return None
    df.columns = [str(c).strip() for c in df.columns]
    cols = list(df.columns)
    time_col = detect_col(cols, [lambda c: any(k in c.lower() for k in ['time','timestamp','date','hour','hr','duration','elapsed'])])
    t = get_time_hours(df[time_col]) if time_col in df.columns else pd.Series([np.nan]*len(df))
    coll_col = detect_col(cols, [lambda c: any(x in c.lower() for x in ['collagen','titre','titer','hydroxyproline','total protein','mg/l','g/l'])])
    return pd.DataFrame({'time_hours': t,
                         'collagen': pd.to_numeric(df[coll_col], errors='coerce') if coll_col in df.columns else np.nan})

def parse_feeding(xf: ExcelFile):
    feed_sheets = [s for s in xf.sheet_names if 'feed' in s.strip().lower()]
    if not feed_sheets: return None
    totals = []
    for sh in feed_sheets:
        try:
            df = xf.parse(sh)
        except Exception:
            continue
        if df.empty: continue
        cand = [c for c in df.columns if re.search(r'(total|used|consumed|feed)', str(c), flags=re.IGNORECASE)]
        if cand:
            sub = pd.to_numeric(df[cand], errors='coerce')
            val = np.nansum(sub.values)
            if np.isfinite(val) and val>0: totals.append(val)
        else:
            coerced = df.apply(pd.to_numeric, errors='coerce')
            val = np.nansum(coerced.values)
            if np.isfinite(val) and val>0: totals.append(val)
    if not totals: return None
    return float(np.nanmax(totals))

def merge_asof_per_file(base: pd.DataFrame, add: pd.DataFrame, cols, tol=0.5):
    if base.empty or add is None or add.empty:
        return base
    l = base.dropna(subset=['time_hours']).sort_values('time_hours')
    r = add.dropna(subset=['time_hours']).sort_values('time_hours')
    if l.empty or r.empty:
        return base
    m = pd.merge_asof(l, r[['time_hours'] + cols], on='time_hours', direction='nearest', tolerance=tol)
    return m


In [64]:
# === Extraction & Merge (run cell) ===
PRIORITY = []
PROCESS_LIMIT = None
excels = iter_excels(EXTRACT_DIR, PRIORITY, PROCESS_LIMIT)
runs = []
for p in excels:
    try:
        xf = ExcelFile(p)
    except Exception:
        continue
    g = parse_growth(xf)
    d = parse_data(xf)
    h = parse_hplc(xf)
    c = parse_compilation(xf, p.name)
    feed_total = parse_feeding(xf)
    if g is None:
        continue
    df = g.copy()
def merge_asof_per_file(base, add, cols, tol=0.1):
    if add is None:
        return base

    l = base.copy()
    r = add.copy()

    # If time_hours is missing we just skip
    if 'time_hours' not in l.columns or 'time_hours' not in r.columns:
        return base

    # 🔑 Make sure both sides use the same dtype
    l['time_hours'] = pd.to_numeric(l['time_hours'], errors='coerce').astype('float64')
    r['time_hours'] = pd.to_numeric(r['time_hours'], errors='coerce').astype('float64')

    # Drop rows where time is NaN
    l = l.dropna(subset=['time_hours'])
    r = r.dropna(subset=['time_hours'])

    # merge_asof requires sorted keys
    l = l.sort_values('time_hours')
    r = r.sort_values('time_hours')

    if l.empty or r.empty:
        return base

    m = pd.merge_asof(
        l,
        r[['time_hours'] + cols],
        on='time_hours',
        direction='nearest',
        tolerance=tol,
    )
    return m

In [65]:
from pathlib import Path

out_dir = Path("snapshots")
out_dir.mkdir(exist_ok=True)

snap_path = out_dir / "model_inputs_snapshot_template.csv"
master.to_csv(snap_path, index=False)
snap_path

PosixPath('snapshots/model_inputs_snapshot_template.csv')

In [66]:
from pathlib import Path

# Choose a folder inside your project
out_dir = Path("model_outputs")
out_dir.mkdir(exist_ok=True)

# Save file HERE instead of /mnt/data
report_path = out_dir / "model_report_template.txt"
report_path.write_text('\n'.join(report))

report_path

PosixPath('model_outputs/model_report_template.txt')

In [70]:
from optinome_app import PROJECT_DIR, build_master
master = build_master(PROJECT_DIR)
print("Master shape:", master.shape)



ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
# === Modeling ===
from pathlib import Path

out_dir = Path("reports")
out_dir.mkdir(exist_ok=True)

report_path = out_dir / "model_report_template.txt"
report_path.write_text('\n'.join(report))
report = []
def nnon(col): return int(master[col].notna().sum()) if col in master.columns else 0

targets = [t for t in ['od600','collagen'] if nnon(t)>=25]
features = ['time_hours','do_pv','do_sp','feed_ml_h','do_data','air_slph','agitation_rpm','ph_data','glycerol','methanol','feed_total_used']
present = [f for f in features if f in master.columns and nnon(f)>=25]
report.append(f'Targets: {targets}\n')
report.append(f'Features: {present}\n')

models = {}
for tgt in targets:
    df_t = master.dropna(subset=[tgt]).copy()
    feats = [f for f in present if f in df_t.columns]
    df_t = df_t.dropna(subset=feats, how='all')
    if df_t.shape[0] < 35 or len(feats)<2:
        report.append(f'{tgt}: insufficient rows after cleaning (rows={df_t.shape[0]}, feats={len(feats)}).\n')
        continue
    X = df_t[feats].fillna(0.0).values
    y = pd.to_numeric(df_t[tgt], errors='coerce').values
    mask = np.isfinite(y); X=X[mask]; y=y[mask]
    if len(y)<35:
        report.append(f'{tgt}: insufficient finite rows ({len(y)}).\n')
        continue
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    r2s, maes = [], []
    for tr, te in kf.split(X):
        mdl = Ridge(alpha=1.0).fit(X[tr], y[tr])
        yp = mdl.predict(X[te])
        r2s.append(r2_score(y[te], yp)); maes.append(mean_absolute_error(y[te], yp))
    ridge = Ridge(alpha=1.0).fit(X, y)
    rf = RandomForestRegressor(n_estimators=200, random_state=42).fit(X, y)
    models[tgt] = {
        'features': feats,
        'ridge_coef': list(ridge.coef_),
        'ridge_intercept': float(ridge.intercept_),
        'cv_r2_mean': float(np.mean(r2s)),
        'cv_r2_std': float(np.std(r2s)),
        'cv_mae_mean': float(np.mean(maes)),
        'rf_feature_importances': dict(zip(feats, rf.feature_importances_.tolist())),
        'rows': int(len(y))
    }
    report.append(f'[{tgt}] R2: {np.mean(r2s):.3f} ± {np.std(r2s):.3f} | MAE: {np.mean(maes):.4g} | rows: {len(y)}\n')

# Save report

print('\n'.join(report))
models

Targets: []

Features: ['time_hours', 'do_data', 'air_slph', 'agitation_rpm', 'ph_data', 'feed_total_used']



{}

In [ ]:
# === Visualization ===
if not master.empty:
    m = master.dropna(subset=['time_hours'])
    if 'od600' in m.columns and m['od600'].notna().sum()>3:
        plt.figure(); plt.plot(m['time_hours'], m['od600'], '.', alpha=0.7)
        plt.xlabel('Time (h)'); plt.ylabel('OD600'); plt.title('OD vs Time'); plt.grid(True)
    if 'collagen' in m.columns and m['collagen'].notna().sum()>3:
        plt.figure(); plt.plot(m['time_hours'], m['collagen'], '.', alpha=0.7)
        plt.xlabel('Time (h)'); plt.ylabel('Collagen'); plt.title('Collagen vs Time'); plt.grid(True)

**Artifacts**  
- `/mnt/data/model_inputs_snapshot_template.csv`  
- `/mnt/data/model_report_template.txt`

# Suggestions & Optimization

Data-driven levers from your models + literature-backed ranges for *Pichia pastoris* AOX1 induction.

In [ ]:
# ==== SUGGESTIONS & OPTIMIZATION ====
import numpy as np, pandas as pd
print("=== Suggestions & Optimization for LivenProteins (OD600 & Collagen) ===\n")
try:
    _models = models
except NameError:
    _models = {}
def top_features(mod, k=5):
    if not mod: return []
    if "rf_feature_importances" in mod and mod["rf_feature_importances"]:
        items = list(mod["rf_feature_importances"].items())
        return [f"{a}: {b:.3f}" for a,b in sorted(items, key=lambda x: -x[1])[:k]]
    elif "features" in mod and "ridge_coef" in mod:
        import numpy as _np
        items = list(zip(mod["features"], _np.abs(mod["ridge_coef"])))
        return [f"{a}: {b:.3f}" for a,b in sorted(items, key=lambda x: -x[1])[:k]]
    return []
for target in ("od600","collagen"):
    if target in _models:
        m=_models[target]
        cv=m.get('cv_r2_mean',None); std=m.get('cv_r2_std',0.0); rows=m.get('rows','?')
        if cv is not None: print(f"[Model insights] {target.upper()}\n - CV R2 ≈ {cv:.3f} (±{std:.3f}) | rows={rows}")
        tf=top_features(m,6)
        if tf: print(" - Top drivers:", ", ".join(tf))
        print()
print("=== Recommended set-points / strategies ===")
print("• DO setpoint: 25–35% during induction; avoid <15%.")
print("• Temperature: 22–28°C during induction (try 25–26°C).")
print("• pH: 5.2–5.8 (start 5.5).")
print("• Agitation/Air: maintain DO (increase airflow first, then rpm).")
print("• Methanol: ramp to residual 0.5–2.0 g/L; adapt after glycerol is depleted.")
print("• Co-feed: sorbitol 20–50% of carbon during induction.")
print("• Phases: glycerol batch → derepression → MeOH adaptation → induction.")
print("• Residuals: glycerol ~0 g/L in induction; methanol steady in target band.\n")
if 'master' in globals():
    import numpy as _np
    if {'glycerol','methanol'}.issubset(set(master.columns)):
        g = master['glycerol'].dropna(); m = master['methanol'].dropna()
        if len(g): print(f"Observed glycerol: {g.min():.3g}–{g.max():.3g} (median {g.median():.3g})")
        if len(m): print(f"Observed methanol: {m.min():.3g}–{m.max():.3g} (median {m.median():.3g})"); print()
print("=== Next-run recipe (edit) ===")
print("A) Glycerol batch: OD600 50–100 @ 28–30°C, pH 5.5, DO>35%.")
print("B) Derepression: 2–4 h low glycerol feed, DO~30%, 26–28°C; deplete glycerol.")
print("C) MeOH adaptation: ramp ~0.5 g/L/h to residual 0.5–1.0 g/L (2–4 h).")
print("D) Induction (18–48 h): DO 25–35%, 25–26°C, pH 5.5; hold MeOH 0.5–2.0 g/L; sorbitol 20–40% carbon.\n")
print("=== DoE screening (3×3×3 + co-feed) ===")
print("Factors: DO 25/30/35%; Temp 22/25/28°C; MeOH 0.5/1.0/2.0 g/L; Sorbitol 0/0.25/0.5.")
print("Responses: collagen titer, STY, OD600, qP, residual MeOH.")
print("QC gates: glycerol ≤0.5 g/L; MeOH oscillations ≤±0.5 g/L; DO never <15% for >10 min; pH within ±0.2.")

=== Suggestions & Optimization for LivenProteins (OD600 & Collagen) ===

=== Recommended set-points / strategies ===
• DO setpoint: 25–35% during induction; avoid <15%.
• Temperature: 22–28°C during induction (try 25–26°C).
• pH: 5.2–5.8 (start 5.5).
• Agitation/Air: maintain DO (increase airflow first, then rpm).
• Methanol: ramp to residual 0.5–2.0 g/L; adapt after glycerol is depleted.
• Co-feed: sorbitol 20–50% of carbon during induction.
• Phases: glycerol batch → derepression → MeOH adaptation → induction.
• Residuals: glycerol ~0 g/L in induction; methanol steady in target band.

=== Next-run recipe (edit) ===
A) Glycerol batch: OD600 50–100 @ 28–30°C, pH 5.5, DO>35%.
B) Derepression: 2–4 h low glycerol feed, DO~30%, 26–28°C; deplete glycerol.
C) MeOH adaptation: ramp ~0.5 g/L/h to residual 0.5–1.0 g/L (2–4 h).
D) Induction (18–48 h): DO 25–35%, 25–26°C, pH 5.5; hold MeOH 0.5–2.0 g/L; sorbitol 20–40% carbon.

=== DoE screening (3×3×3 + co-feed) ===
Factors: DO 25/30/35%; Temp 22